In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from GoogleNews import GoogleNews
from newspaper import Article
from newspaper import Config
from wordcloud import WordCloud, STOPWORDS

In [2]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
now = dt.date.today()
now = now.strftime('%m-%d-%Y')
yesterday = dt.date.today() - dt.timedelta(days = 1)
yesterday = yesterday.strftime('%m-%d-%Y')

In [4]:
nltk.download('punkt')
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'
config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 10

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# save the search term in a variable
search_term = input("Please provide the search term: ")
#As long as the search term is valid not empty...
if search_term != '':
    print(f'Searching for and analyzing {search_term}')

    #Extract News with Google News
    googlenews = GoogleNews(start=yesterday,end=now)
    googlenews.search(search_term)
    result = googlenews.result()
    #store the results
    df = pd.DataFrame(result)
    print(df)

Please provide the search term:  TSLA


Searching for and analyzing TSLA
                                               title               media  \
0  Tesla Motors, Inc. (NASDAQ:TSLA), Facebook, In...      sawahpress.com   
1  Tesla Price Cut Fuels Doubling of Foreign EV S...       Bloomberg.com   
2  Tesla Is Miles Ahead of Others in the EV Race,...            Barron's   
3  Active Stock Evaluation: Tesla, Inc. (NASDAQ:T...       Stocks Equity   
4  Tesla PT raised to $1,300 by Morgan Stanley's ...       Seeking Alpha   
5  Tesla (TSLA) PT Raised to $1,300 at Morgan Sta...   StreetInsider.com   
6                       US stock market news [Video]            FXStreet   
7      Nasdaq ekes out gain in late session comeback      MarketScreener   
8  Tesla signs deal for first U.S. nickel supply ...             Reuters   
9  Tesla signs six-year deal with Talon Metals Co...  Drive Tesla Canada   

          date                   datetime  \
0  4 hours ago 2022-01-10 20:47:54.498912   
1  6 hours ago 2022-01-10 18:47:54.51104

In [6]:
try:
    list =[] #creating an empty list 
    for i in df.index:
        dict = {} #creating an empty dictionary to append an article in every single iteration
        article = Article(df['link'][i],config=config) #providing the link
        try:
            article.download() #downloading the article 
            article.parse() #parsing the article
            article.nlp() #performing natural language processing (nlp)
        except:
            pass 
        #storing results in our empty dictionary
        dict['Date']=df['date'][i] 
        dict['Media']=df['media'][i]
        dict['Title']=article.title
        dict['Article']=article.text
        dict['Summary']=article.summary
        dict['Key_words']=article.keywords
        list.append(dict)
    check_empty = not any(list)
    # print(check_empty)
    if check_empty == False:
        news_df=pd.DataFrame(list) #creating dataframe
        print(news_df)

except Exception as e:
    #exception handling
    print("exception occurred:" + str(e))
    print('Looks like, there is some error in retrieving the data, Please try again or try with a different ticker.' )

          Date               Media  \
0  4 hours ago      sawahpress.com   
1  6 hours ago       Bloomberg.com   
2  6 hours ago            Barron's   
3  8 hours ago       Stocks Equity   
4  8 hours ago       Seeking Alpha   
5  9 hours ago   StreetInsider.com   
6  9 hours ago            FXStreet   
7  9 hours ago      MarketScreener   
8  9 hours ago             Reuters   
9  9 hours ago  Drive Tesla Canada   

                                               Title  \
0                                                      
1                                   Are you a robot?   
2  Tesla Is Miles Ahead of Others in the EV Race,...   
3  Active Stock Evaluation: Tesla, Inc. (NASDAQ:T...   
4  Tesla PT raised to $1,300 by Morgan Stanley's ...   
5  Tesla (TSLA) PT Raised to $1,300 at Morgan Sta...   
6                       US stock market news [Video]   
7      Nasdaq ekes out gain in late session comeback   
8  Tesla signs deal for first U.S. nickel supply ...   
9  Tesla signs six-ye

In [7]:
# Sentiment Analysis
def percentage(part, whole):
    return 100 * float(part) / float(whole)


# Assigning Initial Values
positive = 0
negative = 0
neutral = 0
# Creating empty lists
news_list = []
neutral_list = []
negative_list = []
positive_list = []
neg_score = []
pos_score = []
neu_score = []
comp_score = []
# Iterating over the tweets in the dataframe
for news in news_df['Summary']:
    news_list.append(news)
    analyzer = SentimentIntensityAnalyzer().polarity_scores(news)
    neg = analyzer['neg']
    neu = analyzer['neu']
    pos = analyzer['pos']
    comp = analyzer['compound']
    neg_score.append(neg)
    pos_score.append(pos)
    neu_score.append(neu)
    comp_score.append(comp)

    if neg > pos:
        negative_list.append(news)  # appending the news that satisfies this condition
        negative += 1  # increasing the count by 1
    elif pos > neg:
        positive_list.append(news)  # appending the news that satisfies this condition
        positive += 1  # increasing the count by 1
    elif pos == neg:
        neutral_list.append(news)  # appending the news that satisfies this condition
        neutral += 1  # increasing the count by 1

news_df['Negative_score'] = neg_score
news_df['Positive_score'] = pos_score
news_df['Neutral_score'] = neu_score
news_df['Compound_score'] = comp_score

positive = percentage(positive, len(news_df))  # percentage is the function defined above
negative = percentage(negative, len(news_df))
neutral = percentage(neutral, len(news_df))

# Converting lists to pandas dataframe
news_list = pd.DataFrame(news_list)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)
# using len(length) function for counting
print("Positive Sentiment:", '%.2f' % len(positive_list), end='\n')
print("Neutral Sentiment:", '%.2f' % len(neutral_list), end='\n')
print("Negative Sentiment:", '%.2f' % len(negative_list), end='\n')

Positive Sentiment: 7.00
Neutral Sentiment: 2.00
Negative Sentiment: 1.00


In [8]:
news_df

,Date,Media,Title,Article,Summary,Key_words,Negative_score,Positive_score,Neutral_score,Compound_score
0,4 hours ago,sawahpress.com,,,,[],0.000,0.000,0.000,0.0000
1,6 hours ago,Bloomberg.com,Are you a robot?,Why did this happen?\n\nPlease make sure your ...,Why did this happen?\nPlease make sure your br...,"[review, information, service, loading, sure, ...",0.000,0.230,0.770,0.8065
2,6 hours ago,Barron's,"Tesla Is Miles Ahead of Others in the EV Race,...",Text size\n\nTesla stock rallied to break a fo...,More gains could lie ahead Tuesday after a pri...,"[wall, price, miles, stock, tesla, ahead, nasd...",0.000,0.061,0.939,0.5984
3,8 hours ago,Stocks Equity,"Active Stock Evaluation: Tesla, Inc. (NASDAQ:T...","Tesla, Inc. (NASDAQ:TSLA) with the stream of 3...","Tesla, Inc. (NASDAQ:TSLA) with the stream of 3...","[nasdaqtsla, active, price, evaluation, quarte...",0.000,0.044,0.956,0.6249
4,8 hours ago,Seeking Alpha,"Tesla PT raised to $1,300 by Morgan Stanley's ...","To ensure this doesn’t happen in the future, p...","To ensure this doesn’t happen in the future, p...","[happen, pt, proceeding, q4, adblocker, morgan...",0.053,0.183,0.763,0.6249
5,9 hours ago,StreetInsider.com,"Tesla (TSLA) PT Raised to $1,300 at Morgan Sta...","(Updated - January 10, 2022 6:02 PM EST)Morgan...","(Updated - January 10, 2022 6:02 PM EST)Morgan...","[updated, morgan, price, username, tesla, pt, ...",0.000,0.000,1.000,0.0000
6,9 hours ago,FXStreet,US stock market news [Video],US Stocks Elliott Wave Analysis: Amazon AMZN E...,US Stocks Elliott Wave Analysis: Amazon AMZN E...,"[information, considering, ii, advice, stock, ...",0.000,0.036,0.964,0.1779
7,9 hours ago,MarketScreener,Nasdaq ekes out gain in late session comeback,"(Adds closing price details, commentary)\n\n* ...","Paul Nolte, portfolio manager at Kingsview Inv...","[comeback, sp, points, investors, earlier, nas...",0.049,0.043,0.908,0.1027
8,9 hours ago,Reuters,Tesla signs deal for first U.S. nickel supply ...,The logo of car manufacturer Tesla is seen at ...,REUTERS/Arnd Wiegmann/File PhotoJan 10 (Reuter...,"[supply, plans, minnesota, tesla, talon, deal,...",0.000,0.078,0.922,0.8683
9,9 hours ago,Drive Tesla Canada,Tesla signs six-year deal with Talon Metals Co...,Tesla has signed a six-year deal with Talon Me...,Tesla has signed a six-year deal with Talon Me...,"[supply, agreed, tamarack, tesla, talon, deal,...",0.000,0.059,0.941,0.6486
